In [2]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())




In [3]:
data

Data(edge_index=[2, 4], x=[3, 1])

In [10]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')

In [13]:
from torch_geometric.data import DataLoader
loader = DataLoader(dataset, batch_size=32, shuffle=True)
for batch in loader:
    print(batch)


Batch(batch=[1035], edge_index=[2, 3934], x=[1035, 3], y=[32])
Batch(batch=[1009], edge_index=[2, 3914], x=[1009, 3], y=[32])
Batch(batch=[980], edge_index=[2, 3786], x=[980, 3], y=[32])
Batch(batch=[1147], edge_index=[2, 4298], x=[1147, 3], y=[32])
Batch(batch=[1024], edge_index=[2, 3890], x=[1024, 3], y=[32])
Batch(batch=[1072], edge_index=[2, 4072], x=[1072, 3], y=[32])
Batch(batch=[923], edge_index=[2, 3578], x=[923, 3], y=[32])
Batch(batch=[865], edge_index=[2, 3524], x=[865, 3], y=[32])
Batch(batch=[964], edge_index=[2, 3698], x=[964, 3], y=[32])
Batch(batch=[899], edge_index=[2, 3440], x=[899, 3], y=[32])
Batch(batch=[1086], edge_index=[2, 4390], x=[1086, 3], y=[32])
Batch(batch=[974], edge_index=[2, 3798], x=[974, 3], y=[32])
Batch(batch=[1350], edge_index=[2, 4922], x=[1350, 3], y=[32])
Batch(batch=[1063], edge_index=[2, 4032], x=[1063, 3], y=[32])
Batch(batch=[1110], edge_index=[2, 4158], x=[1110, 3], y=[32])
Batch(batch=[1216], edge_index=[2, 4310], x=[1216, 3], y=[32])
Batc

In [8]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [9]:
data = dataset[0]
data

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

In [30]:
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'],
                    pre_transform=T.KNNGraph(k=6),
                    transform=T.RandomTranslate(0.01))

dataset[0]

Data(category=[1], pos=[2518, 3], y=[2518])

In [20]:
dataset[0]['y']

tensor([0, 0, 3,  ..., 3, 1, 1])

In [33]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')
dataset[0]

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

In [37]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import os

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)
    
os.environ["CUDA_VISIBLE_DEVICES"]='0'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    print('epoch:'+str(epoch)+'    '+'loss:'+str(loss))

/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_15652722793

epoch:0    loss:tensor(1.9580, device='cuda:0', grad_fn=<NllLossBackward>)
epoch:1    loss:tensor(1.8413, device='cuda:0', grad_fn=<NllLossBackward>)
epoch:2    loss:tensor(1.7158, device='cuda:0', grad_fn=<NllLossBackward>)
epoch:3    loss:tensor(1.5607, device='cuda:0', grad_fn=<NllLossBackward>)
epoch:4    loss:tensor(1.4403, device='cuda:0', grad_fn=<NllLossBackward>)
epoch:5    loss:tensor(1.3162, device='cuda:0', grad_fn=<NllLossBackward>)
epoch:6    loss:tensor(1.1771, device='cuda:0', grad_fn=<NllLossBackward>)
epoch:7    loss:tensor(1.0435, device='cuda:0', grad_fn=<NllLossBackward>)
epoch:8    loss:tensor(0.9937, device='cuda:0', grad_fn=<NllLossBackward>)


/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_15652722793

epoch:9    loss:tensor(0.8560, device='cuda:0', grad_fn=<NllLossBackward>)
epoch:10    loss:tensor(0.7724, device='cuda:0', grad_fn=<NllLossBackward>)
epoch:11    loss:tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward>)
epoch:12    loss:tensor(0.6112, device='cuda:0', grad_fn=<NllLossBackward>)


/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_15652722793

...

/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_15652722793

epoch:189    loss:tensor(0.0504, device='cuda:0', grad_fn=<NllLossBackward>)
epoch:190    loss:tensor(0.0338, device='cuda:0', grad_fn=<NllLossBackward>)
epoch:191    loss:tensor(0.0258, device='cuda:0', grad_fn=<NllLossBackward>)
epoch:192    loss:tensor(0.0240, device='cuda:0', grad_fn=<NllLossBackward>)
epoch:193    loss:tensor(0.0374, device='cuda:0', grad_fn=<NllLossBackward>)
epoch:194    loss:tensor(0.0240, device='cuda:0', grad_fn=<NllLossBackward>)


/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_15652722793

epoch:195    loss:tensor(0.0270, device='cuda:0', grad_fn=<NllLossBackward>)
epoch:196    loss:tensor(0.0198, device='cuda:0', grad_fn=<NllLossBackward>)
epoch:197    loss:tensor(0.0299, device='cuda:0', grad_fn=<NllLossBackward>)
epoch:198    loss:tensor(0.0317, device='cuda:0', grad_fn=<NllLossBackward>)
epoch:199    loss:tensor(0.0430, device='cuda:0', grad_fn=<NllLossBackward>)


/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.


In [38]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))


Accuracy: 0.8020


/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.


In [39]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')  # "Add" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3-5: Start propagating messages.
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j, edge_index, size):
        # x_j has shape [E, out_channels]

        # Step 3: Normalize node features.
        row, col = edge_index
        deg = degree(row, size[0], dtype=x_j.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels]

        # Step 5: Return new node embeddings.
        return aggr_out
    
conv = GCNConv(16, 32)
x = conv(x, edge_index)

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 1. Got 4 and 2 in dimension 0 at /opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/TH/generic/THTensor.cpp:689